# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 24 нояб

__Жесткий дедлайн: 27 нояб


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов, 2 балл за доп вопросы, если возникнут, если вопросов не возникло, считаем, что 2 балла вы получили 

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [ ]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

: 

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [83]:
import re
import nltk
from nltk.corpus import words

nltk.download('words')
word_list = set(words.words())

def clean_message(msg):
    msg = re.sub(r'[^\w\s]', '', msg)
    tokens = msg.split()
    filtered_words = [word for word in tokens if word.lower() in word_list]
    return filtered_words
    #return ' '.join(filtered_words)

[nltk_data] Downloading package words to
[nltk_data]     /Users/tatyanagordenko/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [84]:
emails['cleaned_message'] = emails['message'].apply(clean_message)

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

In [95]:
emails['cleaned_message']

0         [Date, Mon, May, From, To, Subject, K, Mail, H...
1         [Date, May, From, To, Subject, Re, K, J, J, Ma...
2         [Date, Wed, From, To, Subject, Re, test, K, Va...
3         [Date, Mon, From, To, Subject, K, L, Gay, mail...
4         [Date, From, To, Subject, Re, Hello, K, Piper,...
                                ...                        
517396    [Date, Wed, PST, From, To, Subject, Trade, wit...
517397    [Date, Wed, PST, From, To, Subject, Gas, Some,...
517398    [Date, Wed, PST, From, To, Subject, RE, CONFID...
517399    [Date, Tue, PST, From, To, Subject, Slone, Ana...
517400    [Date, Mon, PST, From, To, Subject, RE, i, thi...
Name: cleaned_message, Length: 517401, dtype: object

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [86]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        for word in vocabulary:
            self.insert(word)
        
    def insert(self, word: str):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTreeNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        node = self.root
        for char in prefix:
            if char in node.children:
                node = node.children[char]
            else:
                return []

        results = []
        self._dfs(node, prefix, results)
        return results

    def _dfs(self, node: PrefixTreeNode, prefix: str, results: List[str]):
        if node.is_end_of_word:
            results.append(prefix)
        for char, child_node in node.children.items():
            self._dfs(child_node, prefix + char, results)

In [87]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [91]:
from typing import List
from collections import defaultdict

class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        self.word_counts = defaultdict(int)
        self.total_words = 0
        for document in corpus:
            for word in document:
                self.word_counts[word] += 1
                self.total_words += 1

        self.vocabulary = list(self.word_counts.keys())
        self.prefix_tree = PrefixTree(self.vocabulary)

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words = self.prefix_tree.search_prefix(prefix)
        probs = [self.word_counts[word] / self.total_words for word in words]
        return words, probs

In [92]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [93]:
from collections import Counter

class NGramLanguageModel:
    def __init__(self, corpus: List[List[str]], n: int):
        self.n = n
        self.ngram_counts = Counter()
        self.context_counts = Counter()

        for sentence in corpus:
            length = len(sentence)
            for i in range(length):
                for k in range(1, length - i + 1):
                    ngram = tuple(sentence[i:i+k])
                    self.ngram_counts[ngram] += 1
                    if len(ngram) > 1:
                        context = ngram[:-1]
                        self.context_counts[context] += 1

    def get_next_words_and_probs(self, prefix: list) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """

        prefix = tuple(prefix)
        total_count = self.context_counts.get(prefix, 0)
        if total_count == 0:
            return [], []

        candidates = {}
        for ngram in self.ngram_counts:
            if len(ngram) == len(prefix) + 1 and ngram[:-1] == prefix:
                next_word = ngram[-1]
                candidates[next_word] = self.ngram_counts[ngram]

        next_words = []
        probs = []
        for word, count in candidates.items():
            prob = count / total_count
            next_words.append(word)
            probs.append(prob)

        return next_words, probs

In [94]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [96]:
from typing import List, Union

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        suggestions = []

        if isinstance(text, str):
            words = text.strip().split()
        else:
            words = text[:]
        
        if not words:
            return []
        
        last_word = words[-1]
        completions, probs = self.word_completor.get_words_and_probs(last_word)
        if completions:
            max_prob_index = probs.index(max(probs))
            completed_word = completions[max_prob_index]
        else:
            completed_word = last_word
        

        full_words = words[:-1] + [completed_word]
        suggestion = [completed_word]
        n = self.n_gram_model.n
        if n > 1:
            context = full_words[-(n - 1):]
        else:
            context = []
        for _ in range(n_words):
            next_words, next_probs = self.n_gram_model.get_next_words_and_probs(context)
            if not next_words:
                break
            max_prob_index = next_probs.index(max(next_probs))
            next_word = next_words[max_prob_index]
            suggestion.append(next_word)
            if n > 1:
                context = context[1:] + [next_word]
            else:
                context = [next_word]
        
        suggestions.append(suggestion)
        return suggestions

In [97]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [98]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!